In [51]:
#import
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

#load dataset
app_store_reviews = pd.read_excel("CustomerFeedback_reviews - JUPY.xlsx")

#define input and output columns
input_features = ["App", "Store" , "App ID" , "Review ID", "Country", "Version", "Rating", "Date", "Author", "Subject", "Body", "Translated Subject", "Translated Body", 
"Sentiment", "Device", "Language", "OS Version", "Reply URL", "Topics", "Custom Topics", "Tags", "Reply Text", "Reply Date"]

output_features = ["Review Number",	"App Name",	"App Store", "App",	"Store",
"App ID", "Country", "Version", "Rating", "Date", "Author", "Subject", "Body",
"Translated Subject", "Translated Body", "Sentiment", "Label1", "Label2", "Label3", "Label4",
"Product Category 1", "Product Category 2", "Product Category 3", "Product", "Product Code", "Error Code",
"Notes", "HC function", "Review-ID from AIM"]		

#clean the dataset
def clean_text(text):
    # Define common placeholders that represent missing values
    missing_placeholders = ["", "NA", "N/A", "null", "None", "-", "--"]

    # Replace known missing indicators with actual NaN
    if text in missing_placeholders:
        return pd.NA
        
    # Covert everything to String (eg. int, float)
    text = str(text).strip();
    # Convert all characters to lowercase to normalize the text
    text = text.lower()
    
    # Remove URLs (e.g., starting with http or www)
    text = re.sub(r"http\S+|www\S+", "", text)
    
    # Remove all characters except lowercase letters and spaces
    # This eliminates punctuation, numbers, emojis, etc.
    text = re.sub(r"[^a-z\s]", "", text)
    
    # Replace multiple spaces with a single space and trim leading/trailing spaces
    text = re.sub(r"\s+", " ", text).strip()
    
    # Return the cleaned version of the text
    return text








In [50]:
# applying the cleantext function to all of the columns
for col in app_store_reviews.columns :
    app_store_reviews[col] = app_store_reviews[col].apply(clean_text)


In [45]:
# showing the first 10 rows of the cleaned version of the imported dataset
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # Don't cut off wide tables

app_store_reviews.head(10)


,Review Number,App Name,App Store,App,Store,App ID,Country,Version,Rating,Date,Author,Subject,Review ID,Body,Translated Subject,Translated Body,Sentiment,Device,Language,OS Version,Reply URL,Topics,Custom Topics,Tags,Label1,Label2,Label3,Label4,Product Category 1,Product Category 2,Product Category 3,Product,Product Code,Error Code,Notes,HC function,Unnamed: 36,Dish Care,Laundry Care,Cooking,PCP,Cooling,Review-ID from AIM,Unnamed: 43,Unnamed: 44
0,1,Home Connect,Google Play,Home Connect,Google Play,com.bshg.homeconnect.android.release,English,7.1.0,1,2021-01-01,Kai Harrekilde-Petersen,NaN,gp:AOqpTOGAFE_yBuhsE0sNOtc4jUXX-lQ2bIcoq2oA-BG...,AVOID LIKE THE PLAGUE. Should have been zero s...,NaN,NaN,negative,OnePlus 8 IN2010/IN2013,English,30,https://app.appbot.co/apps/2518608-home-connec...,"[""Connectivity""]","[""Connection & errors""]",NaN,Sporadic connection losses,NaN,NaN,NaN,Laundry Care,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Home Connect,Google Play,Home Connect,Google Play,com.bshg.homeconnect.android.release,Polish,7.1.0,2,2021-01-01,Maria Kędziora,NaN,gp:AOqpTOFqQWSLFgMovs6Y7Dwvuo4XkPhNueKt0B5HN3I...,Bardzo słabo się łączy z telefonem. U mnie WFI...,NaN,It connects very poorly with the phone. For me...,negative,Samsung Galaxy Note9 SM-N960F,Polish,29,https://app.appbot.co/apps/2518608-home-connec...,[],[],NaN,Connection speed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Home Connect,Google Play,Home Connect,Google Play,com.bshg.homeconnect.android.release,English,7.1.0,1,2021-01-01,Tom Zgaj,NaN,gp:AOqpTOFfh1SS85wRwwYlur-ef9fh1V5XOP5vzPZQGa4...,Difficult to add appliance. Restarting app aft...,NaN,NaN,negative,Samsung Galaxy Note9 SM-N960F,English,29,https://app.appbot.co/apps/2518608-home-connec...,"[""Design & UX"", ""Customer Support""]",[],NaN,App reliability,Pairing,NaN,NaN,Laundry Care,NaN,NaN,Washer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Home Connect,Google Play,Home Connect,Google Play,com.bshg.homeconnect.android.release,German,NaN,1,2021-01-01,Dieter Willecke,NaN,gp:AOqpTOHsR8_lMnEK8XLxVK-zN9-Xrn9gajjBF-g3Ga3...,Lässt sich nicht mit home connect verbinden (F...,NaN,Cannot be connected to home connect (Fold 2 / ...,negative,Samsung Galaxy Fold/Galaxy Fold 5G,German,29,https://app.appbot.co/apps/2518608-home-connec...,"[""Connectivity"", ""Dissatisfied users""]","[""Connection & errors""]",NaN,Pairing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Home Connect App,iOS,Home Connect App,iOS,901397789,United Arab Emirates,7.1.0,5,2021-01-01,AFosterJX,Do you wanna raise downloads and revenue over ...,6818825526,Please contact me at Skype/WhatsApp: +86189035...,NaN,NaN,mixed,NaN,English,NaN,https://app.appbot.co/apps/598919-home-connect...,"[""Satisfied users"", ""Customer Support"", ""Update""]",[],NaN,Spam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Home Connect,Google Play,Home Connect,Google Play,com.bshg.homeconnect.android.release,German,7.1.0,3,2021-01-01,Thomas Hagen,NaN,gp:AOqpTOHq4X9nuVaJ4EG3P7mUwa0mMSy1rDte5__W3AZ...,Verbindung zum Gerät funktioniert nicht immer....,NaN,Connection to the device does not always work....,negative,Google Pixel 3,German,30,https://app.appbot.co/apps/2518608-home-connec...,"[""Bugs"", ""Connectivity""]","[""Connection & errors""]",NaN,Sporadic connection losses,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Home Connect,Google Play,Home Connect,Google Play,com.bshg.homeconnect.android.release,Greek,7.1.0,5,2021-01-01,Fragiskos Kefalas,NaN,gp:AOqpTOED-gxUKxzcPPTlNxbtJNVHm-TjzHXYRf_iYYd...,"πολύ καλή εφαρμογή, απομακρυσμένος χειρισμός κ...",NaN,"very good application, remote operation and in...",positive,Redmi Note 8 Pro,Greek,29,https://app.appbot.co/apps/2518608-home-connec...,[],[],NaN,Remote Start/Control,NaN,NaN,NaN,Laundry Care,NaN,NaN,Washer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na